In [1]:
import numpy as np
import pandas as pd
from os import path as op
import glob
import random
from copy import deepcopy
from PIL import Image
from scipy import stats

In [3]:
from skimage.io import imread, imshow
from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv

In [4]:
maindir = '/Users/ShariLiu/Dropbox (MIT)/Research/Studies/_NIRS/NES/exp2/analysis_scripts/vis_statistics'
#videos = glob.glob(op.join(maindir, 'videos/*.mp4'))
videos = glob.glob(op.join(maindir, 'stills/*'))

In [5]:
x = videos[1]
nameofx = op.basename(x).split('.mp4')[0]
op.join(maindir, 'stills', nameofx)

'/Users/ShariLiu/Dropbox (MIT)/Research/Studies/_NIRS/NES/exp2/analysis_scripts/vis_statistics/stills/physics_support-exp1_ball_unexp'

In [6]:
# INITIALIZE OUTPUT DATAFRAMES 
all_videos_df = pd.DataFrame()
contr_all_videos_df = pd.DataFrame()

# luminance calculated following iCatcher+ visualize.py function 'sample_luminance': 
# https://github.com/yoterel/icatcher_plus/blob/a52a3ecb5d02de94098f6c1a22d724885db66dcb/visualize.py

for vI in range(len(videos)):
    # INITIALIZE VECTORS FOR EACH VIDEO
    lum_means = [];
    contrasts = [];

    # LOAD VIDEO AND SPLIT FILENAME INTO COND INFO
    videofile = videos[vI];
    video_name = op.basename(videofile).split('.mp4')[0]
    stillpath = op.join(maindir, 'stills', video_name)
    stills = glob.glob(op.join(stillpath, '*.jpg'))
    name_parts = video_name.split('_')
    
    # GET APPROPRIATE COND NAMES ------------------------------------------------------------------

    domain = name_parts[0]
    scenario_group = name_parts[1]
    scenario_specific = name_parts[2]
    event = name_parts[3]

    print('current video: ' + video_name)

    # for each frame in this video.. 
    for imI in range(0, len(stills), 2):
        
        # open the video 
        im = Image.open(stills[imI])
        # split into R-G-B
        imsplit = Image.Image.split(im)
        r = np.array(imsplit[0])
        g = np.array(imsplit[1])
        b = np.array(imsplit[2])
        
        # ----- LUMINANCE -----
        b = (b / 255) ** 2.2
        g = (g / 255) ** 2.2
        r = (r / 255) ** 2.2
        lum_image = 0.2126 * r + 0.7152 * g + 0.0722 * b
        
        # ------ CONTRAST ON GREYSCALE IMG ----- 
        skimg_grey = rgb2gray(imread(stills[imI]))
        contrast_arr = np.ravel(np.array(skimg_grey))
        iqr_contrast = stats.iqr(contrast_arr)
        luminance_out = np.mean(lum_image)
        
        # append to list per-video
        lum_means.append([domain, scenario_group, scenario_specific, event, stills[imI].split('frame_')[1].split('.jpg')[0], luminance_out])
        contrasts.append([domain, scenario_group, scenario_specific, event, stills[imI].split('frame_')[1].split('.jpg')[0], iqr_contrast])
        
        # close image
        im.close()
        
        #entropy_img = entropy(skimg_grey, disk(5))
        #entropy_list.append([domain, task_specific, scenario_string, event, stills[imI].split('frame_')[1].split('.jpg')[0], np.mean(entropy_img)])
    
    # save per video files for frame-wise luminance, contrast
    outpath_lum_means = 'outputs/luminance_perframe_' + video_name + '.csv'
    lum_means_df = pd.DataFrame(lum_means, columns=
                                ['domain', 'scenario_group', 'scenario_specific', 'event','frame_num', 'mean_luminance'])
    lum_means_df.to_csv(outpath_lum_means, index=False) 
    
    outpath_contrasts = 'outputs/contrast_perframe_' + video_name + '.csv'
    contrasts_df = pd.DataFrame(contrasts, columns=
                                ['domain', 'scenario_group', 'scenario_specific', 'event','frame_num', 'iqr_contrast'])
    contrasts_df.to_csv(outpath_contrasts, index=False)
    
    # get mean across frames 
    video_mean_luminance = np.mean(lum_means_df['mean_luminance'])
    video_mean_iqr_contrast = np.mean(contrasts_df['iqr_contrast'])
    
 
 # save out per video mean contrast/luminance  
    vidmean_df = pd.DataFrame({
        'domain': domain,
        'scenario_group': scenario_group, 
        'scenario_specific': scenario_specific, 
        'event': event,
        'video': video_name, 
        'mean_luminance': video_mean_luminance}, index=[0])
    all_videos_df = all_videos_df.append(vidmean_df)
    
    
    vidmean_df_contr = pd.DataFrame({
        'domain': domain,
        'scenario_group': scenario_group, 
        'scenario_specific': scenario_specific, 
        'event': event,
        'video': video_name, 
        'iqr_contrast': video_mean_iqr_contrast}, index=[0])
    contr_all_videos_df = contr_all_videos_df.append(vidmean_df_contr)
# all_videos_df.to_csv('outputs/LUMINANCE_ALL_VIDEOS.csv', index=False)
# contr_all_videos_df.to_csv('outputs/CONTRAST_ALL_VIDEOS.csv', index=False)

all_videos_df.to_csv('outputs/LUMINANCE_SUPPORT.csv', index=False)
contr_all_videos_df.to_csv('outputs/CONTRAST_SUPPORT.csv', index=False)
    
    

current video: physics_support-exp1_ball_fam
current video: physics_support-exp1_ball_unexp
current video: physics_support-exp1_ball_exp


In [7]:
print('done!')

done!
